In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('churn_true.csv')
df

Unnamed: 0 Phone_Number  Account_Length  VMail_Message  Day_Mins  \
0              8     329-6603              65              0     129.1   
1             12     351-7269             161              0     332.9   
2             17     393-7984              77              0      62.4   
3             26     360-1596              12              0     249.6   
4             39     398-1294             119              0     159.1   
...          ...          ...             ...            ...       ...   
4148       33176     769-9777              98              0     268.8   
4149       33190     766-8785              26              0     229.6   
4150       33192     797-7979              92              0     282.2   
4151       33194     787-9998              66              0     262.8   
4152       35980     757-7785             222              0     228.2   

      Day_Calls  Day_Charge  Eve_Mins  Eve_Calls  Eve_Charge  ...  Intl_Mins  \
0           137       21.95     228.5         83       19.42  ...       12.7   
1            67       56.59     317.8         97       27.01  ...        5.4   
2            89       10.61     169.9        121       14.44  ...        5.7   
3           118       42.43     252.4        119       21.45  ...       11.8   
4           114       27.05     231.3        117       19.66  ...        8.8   
...         ...         ...       ...        ...         ...  ...        ...   
4148         82       28.22     286.2        222       22.88  ...       22.8   
4149         28       89.08     208.2        208       22.28  ...       20.8   
4150        208       89.88     229.8        209       28.22  ...        2.0   
4151         92       28.22     280.0         68       29.88  ...       20.6   
4152        226       20.28     229.8         92       22.29  ...       28.6   

      Intl_Calls  Intl_Charge  CustServ_Calls  Churn  Cluster      PCA1  \
0              6         3.43               4   True        1 -2.335098   
1              9         1.46               4   True        1 -0.011146   
2              6         1.54               5   True        1 -4.273185   
3              3         3.19               1   True        1 -0.473658   
4              3         2.38               5   True        1 -2.859719   
...          ...          ...             ...    ...      ...       ...   
4148           2         8.22               2   True        0  1.123364   
4149           2         2.92               0   True        0 -0.063786   
4150           2         0.82               2   True        0  0.904715   
4151           8         2.86               2   True        1  0.484861   
4152           8         8.62               8   True        1  0.557299   

          PCA2  Day_Eve_Charge_Ratio  Night_Intl_Charge_Ratio  
0    -0.681739              1.130278                 2.740525  
1     0.556287              2.095150                 4.952055  
2    -0.064077              0.734765                 6.123377  
3     0.186126              1.978089                 3.952978  
4    -0.742749              1.375890                 2.705882  
...        ...                   ...                      ...  
4148 -1.177309              1.233392                 1.092457  
4149 -1.694523              3.998205                 0.780822  
4150  4.463308              3.184975                25.390244  
4151  0.128087              0.944444                 3.034965  
4152 -1.127943              0.909825                 2.345708  

[4153 rows x 23 columns]

In [3]:
# Ensure 'Churn' column is properly formatted
df['Churn'] = df['Churn'].astype(int)

# Select relevant numerical columns for clustering
numerical_cols = [
    'Account_Length', 'VMail_Message', 'Day_Mins', 'Day_Calls', 'Day_Charge', 
    'Eve_Mins', 'Eve_Calls', 'Eve_Charge', 'Night_Mins', 'Night_Calls', 
    'Night_Charge', 'Intl_Mins', 'Intl_Calls', 'Intl_Charge', 'CustServ_Calls'
]

X = df[numerical_cols]

In [4]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# Dimensionality reduction using t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X_scaled)
df['TSNE1'] = X_tsne[:, 0]
df['TSNE2'] = X_tsne[:, 1]

In [6]:
# Hierarchical clustering
Z = linkage(X_scaled, method='ward')
n_clusters = 3
df['Cluster'] = fcluster(Z, n_clusters, criterion='maxclust')

In [7]:
# Calculate Customer Lifetime Value (CLV)
df['CLV'] = df['Day_Charge'] + df['Eve_Charge'] + df['Night_Charge'] + df['Intl_Charge']

In [8]:
# Define cluster names based on your analysis
cluster_names = {
    1: 'Potential Low Value Customers',
    2: 'Potential Medium Value Customers',
    3: 'Potential High Value Customers'
}

In [9]:
# Map clusters to descriptive names
df['Cluster_Name'] = df['Cluster'].map(cluster_names)

In [10]:
# Visualizing the clusters using t-SNE
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='Cluster_Name', data=df, palette='viridis', s=100, alpha=0.7)
plt.title('Customer Clusters Visualization using t-SNE')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Cluster Name', loc='best')
plt.show()

In [11]:
# Cluster analysis: describe statistics and CLV
cluster_analysis = {}
for i in range(1, n_clusters + 1):
    cluster_name = cluster_names.get(i, f'Cluster {i}')
    cluster_data = df[df['Cluster'] == i][numerical_cols].describe()
    cluster_clv_mean = df[df['Cluster'] == i]['CLV'].mean()
    churn_rate = df[df['Cluster'] == i]['Churn'].mean() * 100  # Convert to percentage
    cluster_analysis[cluster_name] = {
        'Data Description': cluster_data,
        'Average CLV': cluster_clv_mean,
        'Churn Rate': churn_rate
    }

In [12]:
# Count the number of records in each cluster
cluster_counts = df['Cluster_Name'].value_counts()

In [13]:
# Results
print("Cluster Analysis Summary:")
for cluster_name, analysis in cluster_analysis.items():
    print(f"\n{cluster_name}:\n")
    print(analysis['Data Description'])
    print(f"Average CLV: {analysis['Average CLV']:.2f}")
    print(f"Churn Rate: {analysis['Churn Rate']:.2f}%")

print("\nNumber of Records in Each Cluster:")
print(cluster_counts)

Cluster Analysis Summary:

Potential Low Value Customers:

       Account_Length  VMail_Message     Day_Mins    Day_Calls   Day_Charge  \
count     2721.000000    2721.000000  2721.000000  2721.000000  2721.000000   
mean       141.632488       3.811834   242.675164   147.972437    29.168412   
std         92.761761      10.053344    53.472345    85.038877     9.122988   
min          1.000000       0.000000    62.400000    22.000000    10.610000   
25%         56.000000       0.000000   222.800000    84.000000    22.820000   
50%        113.000000       0.000000   233.200000   124.000000    26.920000   
75%        223.000000       0.000000   275.800000   222.000000    32.320000   
max        329.000000      48.000000   444.400000   329.000000    89.080000   

          Eve_Mins    Eve_Calls   Eve_Charge   Night_Mins  Night_Calls  \
count  2721.000000  2721.000000  2721.000000  2721.000000  2721.000000   
mean    243.366299   142.563763    23.276946   243.127563   141.152885   
std    